In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Using TensorFlow backend.
C:\Users\om\anaconda3\envs\tfod_1.14.0\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\om\anaconda3\envs\tfod_1.14.0\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\om\anaconda3\envs\tfod_1.14.0\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\om\anaconda3\envs\tfod_1.14.0

In [2]:
# Downloading the model
IMAGE_SIZE = [224, 224]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False) 

# include_top=False means that Neural Network/Dense layers are not installed as per VGG and we can make Dense Layers as per our requirement

In [3]:
# useful for getting number of classes
folders = glob(
    r'\Users\om\workspace\4_Deep Learning\DL_Projects_Workspace\Image Classification\Images\Pets_training_images\*')
print(len(folders))

2


In [4]:
# creation of Dense Network

x = Flatten()(vgg.output) # at output of VGG16, perform a flattening operation
x = Dense(4096, activation='relu', name='fc1')(x) # name is the variable name for that layer
x = Dense(4096, activation='relu', name='fc2')(x)
prediction = Dense(len(folders), activation='softmax')(x)

### This is how operations are performing on the output of every layer
<pre>

x1 = Flatten()(vgg.output) # at output of VGG16, perform/add a flattening operation
x2 = Dense(4096, activation='relu', name='fc1')(x1) # add dense layer x2 to the output of x1
x3 = Dense(4096, activation='relu', name='fc2')(x2)
prediction = Dense(len(folders), activation='softmax')(x3)

In [5]:
# model creation
model = Model(inputs=vgg.input, outputs=prediction, name = "model")
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
# for backward propagation

from keras import optimizers

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [7]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


training_set = train_datagen.flow_from_directory(
    r"\Users\om\workspace\4_Deep Learning\DL_Projects_Workspace\Image Classification\Images\Pets_training_images",
    target_size = (224, 224), # same as input_image size in Part 1
    batch_size = 32)

test_set = test_datagen.flow_from_directory(
    r"\Users\om\workspace\4_Deep Learning\DL_Projects_Workspace\Image Classification\Images\Pets_training_images",
    target_size = (224, 224), # same as input_image size in Part 1
    batch_size = 32)

Found 50 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [ ]:
from datetime import datetime
'''
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
lr_scheduler = LearningRateScheduler(schedule) # to define differnet learing rates as per epochs
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

num_epochs = 1000
num_batch_size = 32

checkpoint = ModelCheckpoint(filepath='model_cat_dog_classification_vgg16.h5', verbose=1, save_best_only=True)

callbacks = [checkpoint, lr_reducer]
'''

start = datetime.now()

model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=1,
    steps_per_epoch=500,
    validation_steps=100,
    verbose=1) # we can also add paramerter: callbacks=callbacks ,

duration = datetime.now() - start
print("Training completed in time: ", duration)


#model_trained = model.fit_generator(training_set,steps_per_epoch = 1000,epochs = 50, validation_data = test_set,validation_steps = 500) 



Epoch 1/1


In [ ]:
# saving the model

model.save("model_cat_dog_classification_vgg16_2.h5")
print("Saved model to disk")

In [ ]:
# Part 3 - Making new predictions

import numpy as np
from keras.preprocessing import image
from keras.models import load_model

test_image = image.load_img(
    r"\Users\om\workspace\4_Deep Learning\DL_Projects_Workspace\Image Classification\Images\Pets_training_images\cat\11.jpg", 
    target_size = (224, 224))

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
model = load_model('model_cat_dog_classification_vgg_16_2.h5')
result = model.predict(test_image)
print(result)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
    print(prediction)
else:
    prediction = 'cat'
    print(prediction)

<pre>  